<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Transformers/blob/main/Fine_Tuning_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install rouge_score
!pip install evaluate
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 12.1 MB/s 
     |████████████████████████████████| 5.8 MB 34.8 MB/s 
     |████████████████████████████████| 132 kB 16.4 MB/s 
     |████████████████████████████████| 182 kB 40.8 MB/s 
     |████████████████████████████████| 212 kB 64.6 MB/s 
     |████████████████████████████████| 127 kB 70.1 MB/s 
     |████████████████████████████████| 7.6 MB 55.1 MB/s 
     |████████████████████████████████| 1.3 MB 51.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=44f0582086b6afcbc6424bb4542bca27a2f155808689e20d5ba5abba1adb892c
  Stored in directo

In [ ]:
from datasets import load_dataset

french_dataset = load_dataset("amazon_reviews_multi", "fr")
english_dataset = load_dataset("amazon_reviews_multi", "en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [ ]:
samples=english_dataset["train"].shuffle().select(range(3))
samples[:]

{'review_id': ['en_0500947', 'en_0936755', 'en_0566128'],
 'product_id': ['product_en_0662500',
  'product_en_0010983',
  'product_en_0539890'],
 'reviewer_id': ['reviewer_en_0440709',
  'reviewer_en_0960628',
  'reviewer_en_0555246'],
 'stars': [3, 5, 1],
 'review_body': ['It works great but sometimes it looses connection to the outside weather sensor also when it come to walls the sensor can not be very far alway from the digital sensor because otherwise it will loose connection',
  'Exactly what I wanted.',
  'This product was somewhat dried out and irritated my skin. I had to toss.'],
 'review_title': ['Works well', 'Five Stars', 'Allergic Reaction'],
 'language': ['en', 'en', 'en'],
 'product_category': ['home', 'office_product', 'beauty']}

In [ ]:
samples=french_dataset["train"].shuffle().select(range(3))
samples[:]

{'review_id': ['fr_0807573', 'fr_0065481', 'fr_0559500'],
 'product_id': ['product_fr_0932644',
  'product_fr_0399074',
  'product_fr_0130758'],
 'reviewer_id': ['reviewer_fr_0173526',
  'reviewer_fr_0053907',
  'reviewer_fr_0158784'],
 'stars': [4, 5, 4],
 'review_body': ["plus de batterie sur ma clio 2 dci , je l'ai branché et hop au quart de tour",
  "tres bon produit déjà commandé une premiere fois tres satisfaite merci encore ,je suis stupefaite dans le sens ou il n'y a pas d'attente pour la reception",
  'Pas très esthétique mais fait le job. Le parquet est enfin protégé. Bon rapport qualité prix. J’en ai commandé d’autres.'],
 'review_title': ['impressionnant',
  'merci',
  'Bon produit, bon rapport qualité prix'],
 'language': ['fr', 'fr', 'fr'],
 'product_category': ['automotive', 'home', 'furniture']}

In [ ]:
english_dataset.set_format("pandas")
df_en_reviews=english_dataset["train"][:]
df_en_reviews

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,en_0964290,product_en_0740675,reviewer_en_0342986,1,Arrived broken. Manufacturer defect. Two of th...,I'll spend twice the amount of time boxing up ...,en,furniture
1,en_0690095,product_en_0440378,reviewer_en_0133349,1,the cabinet dot were all detached from backing...,Not use able,en,home_improvement
2,en_0311558,product_en_0399702,reviewer_en_0152034,1,I received my first order of this product and ...,The product is junk.,en,home
3,en_0044972,product_en_0444063,reviewer_en_0656967,1,This product is a piece of shit. Do not buy. D...,Fucking waste of money,en,wireless
4,en_0784379,product_en_0139353,reviewer_en_0757638,1,went through 3 in one day doesn't fit correct ...,bubble,en,pc
...,...,...,...,...,...,...,...,...
199995,en_0046316,product_en_0980158,reviewer_en_0629807,5,"Cute slippers, my MIL loved them.",Nice and fit as advertised,en,shoes
199996,en_0956024,product_en_0954574,reviewer_en_0459072,5,My 6 year old likes this and keeps him engaged...,good to keep the kids engaged,en,toy
199997,en_0589358,product_en_0402982,reviewer_en_0199163,5,Replaced my battery with it. Works like new.,This works,en,wireless
199998,en_0970602,product_en_0873374,reviewer_en_0590563,5,"I like them, holding up well.",Well made.,en,industrial_supplies


In [ ]:
df_en_reviews.product_category.value_counts()[:20]

home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: product_category, dtype: int64

In [ ]:
english_dataset.reset_format()
english_books=english_dataset.filter(lambda x : [cat for cat in x["product_category"] if cat in ["book", "digital_ebook_purchase"]], batched=True)
french_books = french_dataset.filter(lambda x : [cat for cat in x["product_category"] if cat in ["book", "digital_ebook_purchase"]], batched=True)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
french_books

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 9995
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 268
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 254
    })
})

In [ ]:
from datasets import concatenate_datasets, DatasetDict

books_dataset = DatasetDict()

for split in english_books.keys():
  books_dataset[split] = concatenate_datasets([english_books[split], french_books[split]])
  books_dataset[split] = books_dataset[split].shuffle(seed=42)


In [ ]:
books_dataset=books_dataset.filter(lambda x: [title for title in x["review_title"] if len(title.split())>2], batched=True)

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
books_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 11411
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 284
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 294
    })
})

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

model_checkpoint = "google/mt5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
inputs = tokenizer("I love reading about AI")
text = tokenizer.convert_ids_to_tokens( inputs["input_ids"])

In [ ]:
print(inputs)
print(text)

{'input_ids': [336, 3869, 11807, 1388, 32419, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]}
['▁I', '▁love', '▁reading', '▁about', '▁AI', '</s>']


In [ ]:
max_input_length = 512
max_target_length = 30

def preprocess_data(examples):
  inputs= tokenizer(examples["review_body"], truncation=True, max_length=max_input_length)
  labels = tokenizer(examples["review_title"], truncation=True, max_length=max_target_length)
  inputs["labels"] = labels["input_ids"]
  return inputs

tokenized_datasets = books_dataset.map(preprocess_data, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11411
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 284
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 294
    })
})

In [ ]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")
scores = rouge_score.compute(predictions = [generated_summary], references=[reference_summary])
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

# Lead3 baseline

In [ ]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [ ]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]
    return metric.compute(predictions=summaries, references=dataset["review_title"])

In [ ]:
score = evaluate_baseline(books_dataset["validation"], rouge_score)

In [ ]:
score

{'rouge1': 0.1411082323640131,
 'rouge2': 0.07621796467239939,
 'rougeL': 0.1345389221872705,
 'rougeLsum': 0.13566651571039806}

## Fine Tuning mT5

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
tokenized_datasets =tokenized_datasets.remove_columns(books_dataset["train"].column_names)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11411
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 284
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 294
    })
})

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn = data_collator,
    shuffle=True,
    batch_size=8
)
tf_val_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn = data_collator,
    shuffle=False,
    batch_size=8
)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

num_epochs=2
optimizer, schedule =create_optimizer(init_lr=5.6e-5, num_train_steps = len(tf_train_dataset)*num_epochs, num_warmup_steps=0, weight_decay_rate=0.01,)
model.compile(optimizer=optimizer)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="mT5-finetuned-amazon-en-fr", tokenizer=tokenizer
)

/content/mT5-finetuned-amazon-en-fr is already a clone of https://huggingface.co/Thabet/mT5-finetuned-amazon-en-fr. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs = num_epochs, callbacks=[callback])

Epoch 1/2
1426/1426 [==============================] - ETA: 0s - loss: 9.7483

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


1426/1426 [==============================] - 461s 307ms/step - loss: 9.7483 - val_loss: 4.3230
Epoch 2/2
1426/1426 [==============================] - 412s 289ms/step - loss: 6.3937 - val_loss: 3.9780


In [ ]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)


@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )


all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

In [ ]:
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value * 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}